<img src="../../docs/images/DSPy8.png" alt="DSPy7 图片" height="120"/>

### 多智能体 DSPy 程序：引导和聚合多个 `ReAct` 智能体

这是一个关于 DSPy 的快速（有些高级）示例。给定一个困难的问答任务和一个智能体架构（`dspy.ReAct`），如何在不调整提示的情况下获得高分呢？

有许多方法，但这个笔记本展示了一种复杂的策略，DSPy 使得实现这一策略变得非常简单：我们将自动引导五个不同的高效提示给 ReAct，然后优化一个聚合器来结合它们的力量。

通常情况下，使用 DSPy 的代码可能比用英语描述更短，所以让我们直接进入代码。

### 0) 简短总结。

这是在相同目录中的`multi_agent.ipynb`示例的副本，但现在使用的是**Llama3-8b**而不是**GPT-3.5**。

我们将在DSPy中构建一个ReAct代理，该代理在基于检索的问答任务上得分为24%准确率。

然后，我们将使用`BootstrapFewShotWithRandomSearch`对其进行优化，以获得35%的准确率。

接着，我们将构建一个多代理聚合器，它由五种不同优化版本的代理组成。

我们的未优化聚合器将得分为21%。它无法理解任务。因此，我们也将优化聚合器。

最终，我们将得到一个经过优化的多代理系统，该系统在相同任务上获得了惊人的59%准确率。

完成这项工作的核心代码部分可以适应15行DSPy代码，但我们将在下面添加一些简短的解释。

### 1) 设置。

我们将配置语言模型（GPT-3.5）和检索模型（ColBERTv2 检索维基百科）。

In [9]:
import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.hotpotqa import HotPotQA
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

# 使用 VLLM 客户端在四个 GPU 上提供 llama3 服务。请注意，这些 URL 对你来说不起作用；你需要参考文档来设置自己的 VLLM/SGLANG 服务器。
llama3 = dspy.HFClientVLLM(model="meta-llama/Meta-Llama-3-8B-Instruct", port=None, url=["http://future-hgx-3:7411", "http://future-hgx-3:7412", "http://future-hgx-3:7413", "http://future-hgx-1:7414"], max_tokens=500, stop=('\n',))
colbert = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')
dspy.configure(lm=llama3, rm=colbert)

### 2) 加载一些数据。

我们将加载150个训练示例（`trainset`），50个验证和优化示例（`valset`），以及300个评估示例（`devset`）。

In [10]:
# 导入HotPotQA数据集
dataset = HotPotQA(train_seed=1, train_size=200, eval_seed=2023, dev_size=300, test_size=0)

# 从训练集中选择前150个样本，并将其转换为只包含问题的形式
trainset = [x.with_inputs('question') for x in dataset.train[0:150]]

# 从训练集中选择第150到第200个样本，并将其转换为只包含问题的形式
valset = [x.with_inputs('question') for x in dataset.train[150:200]]

# 将开发集中的样本转换为只包含问题的形式
devset = [x.with_inputs('question') for x in dataset.dev]

# 展示一个样本数据点；它只是一个问题-答案对
trainset[0]

Example({'question': 'At My Window was released by which American singer-songwriter?', 'answer': 'John Townes Van Zandt'}) (input_keys={'question'})

### 3) ReAct 代理

我们的代理将是一个 DSPy ReAct 代理，通过使用 ColBERTv2 检索工具，接收一个“问题”并输出一个“答案”。

In [11]:
# 创建一个ReAct代理，定义了一个简单的对话规则"question -> answer"，并指定使用Retrieve工具来检索答案
agent = dspy.ReAct("question -> answer", tools=[dspy.Retrieve(k=1)])

让我们在`devset`上评估这个**未优化**的ReAct代理。

In [19]:
# 在开发集的前300个示例上设置一个评估器。
config = dict(num_threads=8, display_progress=True, display_table=5)
# 创建一个评估器对象，使用answer_exact_match度量方法，配置参数为num_threads=8, display_progress=True, display_table=5
evaluate = Evaluate(devset=devset, metric=dspy.evaluate.answer_exact_match, **config)

# 对agent进行评估
evaluate(agent)

  0%|          | 0/300 [00:00<?, ?it/s]

Average Metric: 72 / 300  (24.0): 100%|██████████| 300/300 [01:37<00:00,  3.07it/s]


,question,example_answer,gold_titles,observations,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","[['Hebei | Hebei (; postal: Hopeh) is a province of China in the North China region. Its one-character abbreviation is ""冀 "" (Jì), named after...","No, Qionghai is not in the Hebei province of China",False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[['2017 NHL Entry Draft | The 2017 NHL Entry Draft was the 55th NHL Entry Draft. The draft was held from June 23–24, 2017, at...","answer=""Vegas Golden Knights""",False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","[['Jay Feaster | Jay Harry Feaster (born July 30, 1962 in Harrisburg, Pennsylvania) is a National Hockey League (NHL) executive currently serving as the Executive...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Æthelweard (son of Alfred)', 'Ealhswith'}","[['Æthelweard (son of Alfred) | Æthelweard (d. 920 or 922) was the younger son of King Alfred the Great and Ealhswith.'], ['Ealhswith | Ealhswith or...",King Alfred the Great,✔️ [True]


24.0

### 4) 优化的 ReAct。

让我们使用 DSPy 的简单 `BootstrapFewShotWithRandomSearch` 优化器来创建 ReAct 程序的成功示例，并尝试使用这些构建的示例来优化提示。将来，我们还可以尝试更复杂的 DSPy 优化器，比如 `MIPRO`。

我们将以这种方式引导 20 个程序。示例将从 `trainset` 开始引导，并在我们的小 `valset` 上进行优化。我们将在之后的 `devset` 上进行评估。

In [22]:
# 创建一个包含配置参数的字典
config = dict(max_bootstrapped_demos=2, max_labeled_demos=0, num_candidate_programs=5, num_threads=8)
# 使用BootstrapFewShotWithRandomSearch类实例化tp对象，传入评估指标和配置参数
tp = BootstrapFewShotWithRandomSearch(metric=dspy.evaluate.answer_exact_match, **config)
# 调用tp对象的compile方法，传入agent、trainset和valset参数，得到优化后的react对象
optimized_react = tp.compile(agent, trainset=trainset, valset=valset)

Average Metric: 2 / 3  (66.7):   4%|▍         | 2/50 [00:00<00:00, 77.81it/s]

Average Metric: 18 / 50  (36.0): 100%|██████████| 50/50 [00:14<00:00,  3.39it/s]


In [35]:
# 调用 evaluate 函数，传入 optimized_react 参数
evaluate(optimized_react)

  0%|          | 0/300 [00:00<?, ?it/s]

Average Metric: 93 / 300  (31.0): 100%|██████████| 300/300 [02:08<00:00,  2.33it/s]


,question,example_answer,gold_titles,observations,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}",[],no,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[[""2017 NHL Expansion Draft | The 2017 NHL Expansion Draft was an expansion draft conducted by the National Hockey League on June 18–20, 2017 to...",George McPhee,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","[['Gretzky (disambiguation) | Wayne Gretzky is a retired National Hockey League player.'], [""2006–07 Detroit Red Wings season | The 2006–07 Detroit Red Wings season was...","""Steve Yzerman""",✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[['Dumfries | Dumfries ( ; possibly from Scottish Gaelic: ""Dùn Phris"" ) is a market town and former royal burgh within the Dumfries and Galloway...",River Nith,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Æthelweard (son of Alfred)', 'Ealhswith'}","[['Ealhswith | Ealhswith or Ealswitha (died 5 December 902) was the wife of King Alfred the Great. Her father was a Mercian nobleman, Æthelred Mucel,...","""Edward the Elder""",False


31.0

In [32]:
# 复制 optimized_react 对象
optimized_reactX = optimized_react.deepcopy()
# 删除 optimized_reactX 对象中的 candidate_programs 属性
del optimized_reactX.candidate_programs

# 配置参数字典
config = dict(max_bootstrapped_demos=2, max_labeled_demos=0, num_candidate_programs=20, num_threads=8)
# 创建 BootstrapFewShotWithRandomSearch 对象 tp
tp = BootstrapFewShotWithRandomSearch(metric=dspy.evaluate.answer_exact_match, **config)
# 使用 tp 对象编译 agent，trainset，valset 和 teacher 参数，返回结果赋值给 optimized_react2
optimized_react2 = tp.compile(agent, trainset=trainset, valset=valset, teacher=optimized_reactX)

In [38]:
# 调用 evaluate 函数，传入 optimized_react2 作为参数
evaluate(optimized_react2)

Average Metric: 105 / 300  (35.0): 100%|██████████| 300/300 [01:10<00:00,  4.27it/s]


,question,example_answer,gold_titles,observations,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","[['Cangzhou | Cangzhou () is a prefecture-level city in eastern Hebei province, People\'s Republic of China. At the 2010 census, Cangzhou\'s built-up (""or metro"") area...",no,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","[[""2017 NHL Expansion Draft | The 2017 NHL Expansion Draft was an expansion draft conducted by the National Hockey League on June 18–20, 2017 to...",George McPhee,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","[['Tony Resch | Tony Resch is a retired lacrosse player, and current field and box lacrosse head coach. He is the former head coach of...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","[[""Crichton Collegiate Church | Crichton Collegiate Church is situated about 0.6 mi south west of the hamlet of Crichton in Midlothian, Scotland. Crichton itself is...",River Tyne,✔️ [True]
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Æthelweard (son of Alfred)', 'Ealhswith'}","[['Steve Ellsworth | Steven Clark Ellsworth (born July 30, 1960 in Chicago) is the son of Dick Ellsworth and is a former Major League Baseball...",answer,False


35.0

### 5) 零射击聚合器。

现在让我们提取最佳的五个引导式 ReAct 程序。我们将构建一个简单的 DSPy 聚合器，运行所有这些程序，然后生成最终答案。

In [49]:
from dsp.utils import flatten, deduplicate

# 从优化过程中得到的性能最佳的五个ReAct程序
AGENTS = [x[-1] for x in optimized_react2.candidate_programs[:5]]

class Aggregator(dspy.Module):
    def __init__(self, temperature=0.0):
        self.aggregate = dspy.ChainOfThought('context, question -> answer')
        self.temperature = temperature

    def forward(self, question):
        # 使用高温运行所有五个代理程序，然后提取并去重它们观察到的上下文
        with dspy.context(lm=gpt3.copy(temperature=self.temperature)):
            preds = [agent(question=question) for agent in AGENTS]
            context = deduplicate(flatten([flatten(p.observations) for p in preds]))

        # 运行聚合步骤以生成最终答案
        return self.aggregate(context=context, question=question)

让我们在优化之前快速评估聚合器。

In [53]:
aggregator = Aggregator()  # 创建一个聚合器实例
evaluate(aggregator)  # 对聚合器进行评估

Average Metric: 64 / 300  (21.3): 100%|██████████| 300/300 [12:53<00:00,  2.58s/it]


,question,example_answer,gold_titles,rationale,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","determine whether both Cangzhou and Qionghai are in the Hebei province of China. We know that Cangzhou is a prefecture-level city in eastern Hebei province,...","No, only Cangzhou is in Hebei province, while Qionghai is not.assistant",False
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}",answer this question. We know that Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season. We also know that the Vegas...,The National Hockey League (NHL).assistant`,False
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}",answer this question. We know that the Wings entered a new era following the retirement of a Canadian retired professional ice hockey player and current...,Steve Yzerman.,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","find the answer. We know that the Crichton Collegiate Church is situated near the hamlet of Crichton in Midlothian, Scotland. We also know that the...",The River Esk.,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Æthelweard (son of Alfred)', 'Ealhswith'}",find the answer. We know that Ealhswith was the wife of King Alfred the Great. We also know that Æthelweard was the younger son of...,King Alfred the Great.,✔️ [True]


21.33

### 6) 优化聚合器。

In [51]:
# 定义关键字参数kwargs
kwargs = dict(max_bootstrapped_demos=2, max_labeled_demos=6, num_candidate_programs=10, num_threads=8)
# 使用BootstrapFewShotWithRandomSearch类实例化对象tp，并传入评估函数和关键字参数kwargs
tp = BootstrapFewShotWithRandomSearch(metric=dspy.evaluate.answer_exact_match, **kwargs)
# 调用tp对象的compile方法，传入聚合器aggregator、训练集trainset和验证集valset，得到优化后的聚合器optimized_aggregator
optimized_aggregator = tp.compile(aggregator, trainset=trainset, valset=valset)

Average Metric: 0 / 1  (0.0):   0%|          | 0/50 [00:00<?, ?it/s]

Average Metric: 26 / 50  (52.0): 100%|██████████| 50/50 [00:15<00:00,  3.30it/s]


In [52]:
# 复制 optimized_aggregator 对象并将其赋值给 optimized_aggregator2
optimized_aggregator2 = optimized_aggregator.deepcopy()
# 将 optimized_aggregator2 对象的 temperature 属性设置为 0.7
optimized_aggregator2.temperature = 0.7

# 调用 evaluate 函数并传入 optimized_aggregator2 对象作为参数
evaluate(optimized_aggregator2)

  0%|          | 0/300 [00:00<?, ?it/s]

Average Metric: 176 / 300  (58.7): 100%|██████████| 300/300 [11:59<00:00,  2.40s/it]


,question,example_answer,gold_titles,rationale,pred_answer,answer_exact_match
0,Are both Cangzhou and Qionghai in the Hebei province of China?,no,"{'Qionghai', 'Cangzhou'}","find the answer. We know that Cangzhou is a prefecture-level city in eastern Hebei province, while Qionghai is one of the seven county-level cities of...",No,✔️ [True]
1,Who conducts the draft in which Marc-Andre Fleury was drafted to the Vegas Golden Knights for the 2017-18 season?,National Hockey League,"{'2017–18 Pittsburgh Penguins season', '2017 NHL Expansion Draft'}","find the answer. We know that Marc-André Fleury was drafted to the Vegas Golden Knights in the 2017 NHL Expansion Draft, and we also know...",The National Hockey League,✔️ [True]
2,"The Wings entered a new era, following the retirement of which Canadian retired professional ice hockey player and current general manager of the Tampa Bay...",Steve Yzerman,"{'2006–07 Detroit Red Wings season', 'Steve Yzerman'}","find the answer. We know that the Wings entered a new era, following the retirement of Steve Yzerman, and we also know that Yzerman is...",Steve Yzerman,✔️ [True]
3,What river is near the Crichton Collegiate Church?,the River Tyne,"{'Crichton Collegiate Church', 'Crichton Castle'}","find the answer. We know that the Crichton Collegiate Church is situated in Midlothian, Scotland, and we also know that the River Esk flows through...",the River Esk,False
4,In the 10th Century A.D. Ealhswith had a son called Æthelweard by which English king?,King Alfred the Great,"{'Æthelweard (son of Alfred)', 'Ealhswith'}","find the answer. We know that Ealhswith was the wife of King Alfred the Great, and we also know that Æthelweard was the younger son...",King Alfred the Great,✔️ [True]


58.67

### 7) 结论。

通常情况下，我们喜欢发布带有预先计算缓存的笔记本，并使用`llama3.inspect_history`来检查提示，以探索优化的行为。请查看介绍笔记本（或自述文件中的任何Colab笔记本）以获取这样的带注释示例！

为了保持当前版本的速度快，如果有足够的兴趣，Omar将把这个笔记本扩展为带注释的版本。